# Libraries

In [1]:
using DataFrames, CSV, Dates, TextAnalysis

In [2]:
using PlotlyJS

WebIO._IJuliaInit()

# Data ImportTextAnalysis

In [3]:
df = CSV.read("Tui_TrustPilot.csv", DataFrame)
select!(df, Not(:Column1))

,Date,Author,Body
,Date,String63,String
1,2020-07-06,Patricia Palmer,Think they need to contact people about all these cancelled holidays and shocking treatment don't tell you anything don't answer the phones or email can't get refunds
2,2020-07-06,Andy Tims,Just atrocious lack of customer service. They send you emails and texts you can’t respond to and it’s just impossible to get through on the phone. \nBeen on holiday with them several times but very much doubt I will again.
3,2020-07-06,Alexis,"If I could do no stars I would.\nHad email to say my holiday cant go ahead, but they wont cancel it so cant get a refund. Have spoken to many people on the phone after holding for an hour esh time only to be hung up on every time. \nDONT TRUST THEM, they will keep hold of your money for as long as they can."
4,2020-07-06,Peter Kerr,"Tui are totally useless, they are not connectable, they don't respond to emails, don't answer the phone. \nSo regret booking a holiday with them!!!"
5,2020-07-06,Charlene,"Shocking customer service, not even a courtesy email to actually tell us our holiday on 14th May is cancelled, waited patiently for a call as requested on the website and still heard diddly squat, called to be told they had to contact the suppliers of the holiday, then told that emails had ‘gone missing’ however after talking to the suppliers myself they have already issued TUI with the refund, so TUI now have our 7500k and are technically with holding it. I was promised I would get it within 3 weeks of the last phone call, upon calling again they’ve changed it to 4... tomorrow is that 4 week deadline and I’m pretty sure I’m not going to blink and see it in my account! THEFT and money laundering springs to mind!!"
6,2020-07-06,Wanda Rutczynska,"This company is the biggest fraud I have come across, ever.\n\nStole my money and impossible to get it back \n\nAVOID AT ALL COSTS"
7,2020-07-06,Rowenah Preger,I don’t think they even qualify for a 1 star. Told us holiday is cancelled and to rebook with £500 incentive - our whole family is going and we can’t because we cannot rebook with the £500 incentive offered. Our holiday hasn’t updated as cancelled on our booking and the numerous calls to TUI and being on hold for over 2 hours since last Friday has got us no where! Nobody can do anything apart from rebooking it at over a £1000 more! Just cancel our booking and rebook it over the phone !!! It’s over £3k terrible customer service !!! They just don’t want to help.
8,2020-07-06,JANINE MURRAY,"TUI Dispicable!! \nIve made numerous calls and finally got through (I think) at 6.45pm tonight, only to be put on hold until I gave up and hung up!! Probably some lazy piece of s*** who didnt want to deal with yet another irate customer!\nThe website is not up to date. I want a full refund but it is offering me only 30%. Why would I be a fool and click on 'accept' ??!! This cannot be legal. I will NEVER book again with Tui and I will make sure that I tell as many people on social media about there shocking service , or lack of!! Hope they go bust."
9,2020-07-06,Jenny Hancock,"Bought my holiday in store September 2019 to Turkey August 2020. No way of getting through to customer services, on phone literally hours trying to get through for information about cancelling the holiday or even some advice if the holiday is even going ahead. Very poor they have not opened the stores. Will not be booking with them again."


## Transform the data

In [4]:
reviews = []
for i in 1:size(df.Body)[1]
    push!(reviews, df.Body[i])
end

title_ = []
for i in 1:size(df.Heading)[1]
    push!(title_, df.Heading[i])
end

author_ = []
for i in 1:size(df.Author)[1]
    push!(author_, df.Author[i])
end

In [5]:
str_doc = []
for i in reviews
    i = StringDocument(i)
    push!(str_doc, i)
end

In [6]:
for i in 1:size(str_doc)[1]
    z = str_doc[i]
    TextAnalysis.title!(z, title_[i])
    author!(z, author_[i])
end

## Preprocessing the data

In [7]:
for i in 1:size(str_doc)[1]
    z = str_doc[i]
    remove_corrupt_utf8!(z)
    remove_case!(z)
    prepare!(z, strip_articles)
    prepare!(z, strip_indefinite_articles)
    prepare!(z, strip_definite_articles)
    prepare!(z, strip_html_tags)
    prepare!(z, strip_numbers)
    prepare!(z, strip_stopwords)
    prepare!(z, strip_non_letters)
    stem!(z)
end

In [8]:
crps = Corpus(str_doc)

A Corpus with 1999 documents:
 * 1999 StringDocument's
 * 0 FileDocument's
 * 0 TokenDocument's
 * 0 NGramDocument's

Corpus's lexicon contains 0 tokens
Corpus's index contains 0 tokens

# The most frequent words

In [9]:
crps[1]

A StringDocument{String}
 * Language: Languages.English()
 * Title: Think they need to contact people about…
 * Author: Patricia Palmer
 * Timestamp: Unknown Time
 * Snippet: contact peopl cancel holiday shock treatment tell 

In [10]:
standardize!(crps, TokenDocument)

In [11]:
update_lexicon!(crps)

In [12]:
freq_words = (stack(DataFrame(lexicon(crps)), names(DataFrame(lexicon(crps)))))
sort!(freq_words, :value, rev=true)

,variable,value
,String,Int64
1,tui,3271
2,holiday,2804
3,book,2194
4,cancel,1539
5,refund,1533
6,custom,1125
7,day,1030
8,call,1011
9,time,978


In [56]:
trace = bar(sort(freq_words[1:20, :], :value), y = :variable, x = :value, orientation = "h", exttemplate="%{text:.2f}", textposition="outside",)

config = PlotConfig(
    toImageButtonOptions=attr(
        format="png", # one of png, svg, jpeg, webp
        filename="custom_image",
        height=1280,
        width=720,
        scale=2 # Multiply title/legend/axis/canvas sizes by this factor
    ).fields,
)

layout = Layout(
    xaxis_title = "Count",
    yaxis_title = "Word",
    title = "Word Frequency",
    font_family = "Arial",
    width = 1280, height = 720
)
plot(trace, layout, config=config)

data: [
  "bar with fields exttemplate, orientation, textposition, type, x, and y"
]

layout: "layout with fields font, height, margin, template, title, width, xaxis, and yaxis"

### Inverse Document Frequemncy

In [26]:
update_lexicon!(crps)

In [27]:
m = DocumentTermMatrix(crps)

A 1999 X 5353 DocumentTermMatrix

In [52]:
matrix_ = dtm(m, :dense)

1999×5353 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0 

In [53]:
tf_idf(matrix_)

1999×5353 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

### Term Frequency

In [50]:
tf(m)

1999×5353 SparseArrays.SparseMatrixCSC{Float64, Int64} with 69142 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠁⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠁

### OKAPI BM-25

In [48]:
bm_25(m)

1999×5353 SparseArrays.SparseMatrixCSC{Float64, Int64} with 69142 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠁⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠁